In [1]:
import os
import pandas as pd

from pathlib import Path
from glob import glob
from datetime import datetime

!pip install xlrd

Pour harmoniser les importations en chemin relatifs, dans des fichiers cousins :

In [2]:
if not "root" in locals():
    current_path = Path(os.getcwd())
    root = current_path.parent.absolute()
    data_folder = str(root) + '/treated_data/'


In [3]:
types = ["xlsx", "csv", "xls"]

os.chdir(data_folder)

sheets_dict = {}

print("-- XLSX FILES --")
for file_path in glob("*.xlsx"):
    file_name = file_path.split(".")[0]


    print(file_name + " sheets:")
    
    file = pd.ExcelFile(file_path)
    for sheet in file.sheet_names:
        print("-->"+sheet)
        sheets_dict[sheet] = file.parse(sheet)
    print()
    
    
print("\n-- XLS FILES --") 
for file_path in glob("*.xls"):
    file_name = file_path.split(".")[0]

    print(file_name + " sheets:")
    
    file = pd.ExcelFile(file_path)
    for sheet in file.sheet_names:
        print("-->"+sheet)
        sheets_dict[sheet] = file.parse(sheet)
    print()
    
print("\n-- csv FILES --")
for file_path in glob("*.csv"):
    file_name = file_path.split(".")[0]
    print(file_name)
    file_df = pd.read_csv(file_path, sep=';')
    sheets_dict[file_name] = file_df


-- XLSX FILES --
Quality Minus Junk Factors Daily sheets:
-->QMJ Factors
-->HML Devil

OGTauxUS sheets:
-->US Generic Govt 10 Yr
-->USD SWAP SEMI 30 360 10Y
-->ICE LIBOR USD 3 Month
-->US Generic Govt 3 Mth
-->USD SWAP OIS 3M

bloom_data_2 sheets:
-->CL1
-->CL2
-->CL3
-->Euribor3
-->Bund10
-->FUTURES MATURITIES

Betting Against Beta Equity Factors Daily sheets:
-->BAB Factors

Volatility_data_merged sheets:
-->volatility


-- XLS FILES --
sentiment sheets:
-->SENTIMENT


-- csv FILES --
F-F_ST_Reversal_Factor_daily
F-F_Research_Data_5_Factors_2x3_daily
F-F_Momentum_Factor_daily


In [4]:
keys = list(sheets_dict.keys())
for key in keys:
    if key !='FUTURES MATURITIES':
        if type(sheets_dict[key].index)!=pd.DatetimeIndex:
            date_col = [col for col in ['Date','DATE','Unnamed: 0','Reported data', 'time'] if col in sheets_dict[key].columns]
            assert len(date_col)==1, "Date column is not unique"+ str( key)
            if key in ['F-F_ST_Reversal_Factor_daily', 'F-F_Research_Data_5_Factors_2x3_daily', 'F-F_Momentum_Factor_daily']:
                sheets_dict[key][date_col]= sheets_dict[key][date_col].applymap(lambda x: datetime.strptime(x,'%d/%m/%Y'))
            sheets_dict[key].set_index(date_col, inplace=True)
            if key == 'VME Factors':
                # Resampling monthly -> daily
                #The value for month n is the one released on the last day of the month n-1, as it is one agents use during month n.
                start_date = sheets_dict["VME Factors"].index.min() - pd.DateOffset(day=1)
                end_date = sheets_dict["VME Factors"].index.max() + pd.DateOffset(day=31)
                dates = pd.date_range(start_date, end_date, freq='D')
                sheets_dict["VME Factors"] = sheets_dict["VME Factors"].reindex(dates, method='ffill')
        sheets_dict[key].index.names = ["Date"]
        #Adding a suffix
        sheets_dict[key].rename(columns = lambda x: x+'_'+key,inplace=True)


In [5]:
principal_key = "volatility"
banned_keys = ['FUTURES MATURITIES']
keys = [e for e in sheets_dict.keys() if not e in [principal_key] + banned_keys]
final_base = pd.merge_asof(sheets_dict[principal_key], sheets_dict[keys[0]], on="Date")

for i in range(1, len(keys)):
    final_base = pd.merge_asof(final_base, sheets_dict[keys[i]], on="Date")

In [6]:
os.chdir(root)

keys_to_remove = ["VOLUME_TOTAL_CALL_VSTOXX_volatility", "PUT_CALL_VOLUME_RATIO_CUR_DAY_VSTOXX_volatility", "PUT_CALL_VOLUME_RATIO_CUR_DAY_VSTOXX_volatility",
                  "TOT_OPT_VOLUME_CUR_DAY_VSTOXX_volatility", "VOLATILITY_360D_VSTOXX_volatility",
                  "HIST_CALL_IMP_VOL_VSTOXX_volatility", "3MTH_IMPVOL_100.0%MNY_DF_VSTOXX_volatility",
                  "3MTH_IMPVOL_105.0%MNY_DF_VSTOXX_volatility", "3MTH_IMPVOL_110.0%MNY_DF_VSTOXX_volatility",
                  "3MTH_IMPVOL_90.0%MNY_DF_VSTOXX_volatility", "3MTH_IMPVOL_95.0%MNY_DF_VSTOXX_volatility", "6MTH_IMPVOL_100.0%MNY_DF_VSTOXX_volatility",
                  "6MTH_IMPVOL_105.0%MNY_DF_VSTOXX_volatility", "6MTH_IMPVOL_110.0%MNY_DF_VSTOXX_volatility", "6MTH_IMPVOL_90.0%MNY_DF_VSTOXX_volatility",
                  "6MTH_IMPVOL_95.0%MNY_DF_VSTOXX_volatility", "12MTH_IMPVOL_100.0%MNY_DF_VSTOXX_volatility", "12MTH_IMPVOL_105.0%MNY_DF_VSTOXX_volatility",
                  "12MTH_IMPVOL_110.0%MNY_DF_VSTOXX_volatility", "12MTH_IMPVOL_90.0%MNY_DF_VSTOXX_volatility", "12MTH_IMPVOL_95.0%MNY_DF_VSTOXX_volatility",
                  "24MTH_IMPVOL_100.0%MNY_DF_VSTOXX_volatility", "24MTH_IMPVOL_105.0%MNY_DF_VSTOXX_volatility", "24MTH_IMPVOL_110.0%MNY_DF_VSTOXX_volatility",
                  "24MTH_IMPVOL_90.0%MNY_DF_VSTOXX_volatility", "24MTH_IMPVOL_95.0%MNY_DF_VSTOXX_volatility", "VOLUME_TOTAL_PUT_VSTOXX_volatility",
                  "OPEN_INT_TOTAL_CALL_VSTOXX_volatility", "OPEN_INT_TOTAL_PUT_VSTOXX_volatility",
                  "12MTH_IMPVOL_100.0%MNY_DF_VIX_volatility", "12MTH_IMPVOL_105.0%MNY_DF_VIX_volatility", "12MTH_IMPVOL_110.0%MNY_DF_VIX_volatility", "12MTH_IMPVOL_90.0%MNY_DF_VIX_volatility", 
                  "12MTH_IMPVOL_95.0%MNY_DF_VIX_volatility", "24MTH_IMPVOL_100.0%MNY_DF_VIX_volatility", "24MTH_IMPVOL_105.0%MNY_DF_VIX_volatility", "24MTH_IMPVOL_110.0%MNY_DF_VIX_volatility", 
                  "24MTH_IMPVOL_90.0%MNY_DF_VIX_volatility", "24MTH_IMPVOL_95.0%MNY_DF_VIX_volatility"]


final_base.set_index("Date").drop(columns=keys_to_remove).to_csv('database.csv', na_rep='#N/A N/A')
